# Test 3 - General Results

### All CUIs and drugs seen in the Drugs_labeled_for_AEs.txt file (synonym CUIs not included) and present in both Version 1 and Version 2 of PathFX

In [1]:
import networkx as nx
import random
from random import sample
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')
import seaborn as sns
from multiprocessing import Pool
import numpy as np
import pandas as pd
import os
import sys
import warnings
import xlrd
import itertools
import pickle
from functools import partial
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib as mpl
mpl.use('TkAgg')
from matplotlib import backend_bases
import matplotlib.pyplot as plt
import itertools
import re
import ast
from sklearn.linear_model import LinearRegression
import plotly
import plotly.express as px
import plotly.graph_objects as go
import dataframe_image as dfi
sns.set_theme()
np.random.seed(0)

In [2]:
"""
Helper Functions
"""

# function checks if directory exists, if not it constructs it
def check_directory_exists(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
        
def read_csv_file(file_path, input_sep=',', input_delimiter=None, input_index_col=None, input_dtype=None, input_delim_whitespace=False, input_low_memory=True):
    return pd.read_csv(filepath_or_buffer=file_path, sep=input_sep, delimiter=None, index_col=input_index_col, dtype=input_dtype, delim_whitespace=input_delim_whitespace, low_memory=input_low_memory)

# funcion saves DataFrame or list to as a textfile
def save_to_csv_file(output_folder_dest, df, file_name, input_index=False, export_style=True, show_print_out=False):
    output_file_name = output_folder_dest + file_name
    output_csv_filename = output_file_name + ".csv"
    output_png_filename = output_file_name + ".png"
    df.to_csv(output_csv_filename, index=input_index)
    style_df = df.copy(deep=True)
    if (export_style):
        #cm = sns.light_palette("seagreen", as_cmap=True)
        #style_df = df.copy(deep=true).style.background_gradient(cmap=cm)
        style_df = df.style.background_gradient()
    try:
        dfi.export(style_df, output_png_filename)
    except ValueError:
        truncated_df = df.copy(deep=True).iloc[0:30][df.columns[0:15]]
        #truncated_style_df = truncated_df.style.background_gradient()
        dfi.export(truncated_df, output_png_filename)
        
    if(show_print_out):
        print("Constructed and saved", output_filename)

# Read in Pickle File
def read_pickle_file(file_path):
    if not os.path.exists(file_path):
        sys.exit("Can't locate input file %s" % file_path)
    return pd.read_pickle(file_path)

In [3]:
# For PathFX Version 1.0
# --------------------------------------------------------------------------
input_filename = "output_database_analysis/cleaned_data_files/cleaned_merged_unique_cuis2genes.pkl"
cleaned_merged_unique_cuis2genes = read_pickle_file(input_filename)

input_filename = "output_database_analysis/cleaned_data_files/cleaned_cuis_to_all_phens.pkl"
cleaned_cuis_to_all_phens = read_pickle_file(input_filename)

input_filename = "output_database_analysis/cleaned_data_files/cleaned_drug_intome_targets.pkl"
cleaned_drugs_to_drugbank_targets = read_pickle_file(input_filename)
# --------------------------------------------------------------------------

# For PathFX Version 2.0
# --------------------------------------------------------------------------
input_filename = "output_database_analysis/cleaned_data_files/cleaned_Pfx050120_merged_unique_cuis2genes.pkl"
cleaned_Pfx050120_merged_unique_cuis2genes = read_pickle_file(input_filename)

input_filename = "output_database_analysis/cleaned_data_files/cleaned_Pfx050120_cui_to_phens.pkl"
cleaned_Pfx050120_cui_to_phens = read_pickle_file(input_filename)

input_filename = "output_database_analysis/cleaned_data_files/cleaned_pfxDB050620_dint.pkl"
cleaned_pfxDB050620_dint = read_pickle_file(input_filename)
# --------------------------------------------------------------------------

In [4]:
parent_folder = "output_modify_all_pathfx_version_drug_association_super_table/"
test = "all_version_intersecting_and_fda_data"
test_folder = parent_folder + test + "/"

# all associated records
test_all_drugs_super_association_df = read_csv_file(test_folder + test + "_all_drugs_super_association_table.csv")
test_all_drugs_super_association_df

,cui,drug,drugbank_ID,phenotype_v1,association_exists_v1,probability_v1,assoc_in_neigh_v1,assoc_in_intom_v1,benjamini_hochberg_v1,neigh_genes_v1,...,drug_targets_v1_size,drug_targets_v2_size,version_difference_drug_target_gene_lst,version_difference_unique_drug_target_gene_lst,version_difference_drug_target_gene_lst_percentage,version_difference_unqiue_drug_target_gene_lst_percentage,version_difference_intersecting_drug_target_gene_lst_percentage,ratio_drug_target_gene_over_neigh_gene_lst_v1_size,ratio_drug_target_gene_over_neigh_gene_lst_v2_size,version_difference_ratio_drug_target_gene_over_neigh_gene_lst_size
0,C0243026,abatacept,DB01281,Sepsis,Yes,1.079882e-06,5,602,0.000060,"['CD80', 'CD86', 'FYN', 'PIK3CA', 'PIK3R1']",...,2,2,0,0,0.000314,0.000000,0.000314,0.400000,0.666667,0.266667
1,C0027051,acamprosate,DB00659,Myocardial Infarction,Yes,2.347811e-09,21,1019,0.000059,"['ADRA1D', 'AGT', 'AVPR1A', 'BDKRB1', 'BDKRB2'...",...,1,2,1,1,0.000144,-0.000419,-0.000419,0.047619,0.117647,0.070028
2,C0038358,acamprosate,DB00659,Gastric ulcer,Yes,4.382202e-09,8,116,0.000063,"['BDKRB1', 'BDKRB2', 'CCK', 'EDN1', 'GAST', 'G...",...,1,2,1,1,0.012656,0.002018,0.002018,0.125000,0.333333,0.208333
3,C0027051,adenosine,DB00640,Myocardial Infarction,Yes,1.735121e-10,36,1019,0.000064,"['ADM', 'ADORA1', 'ADRA2B', 'ADRB1', 'ADRB3', ...",...,6,6,0,0,-0.002514,0.000000,-0.002514,0.166667,0.171429,0.004762
4,C0030305,aldesleukin,DB00041,Pancreatitis,Yes,1.519662e-04,5,277,0.000320,"['CYP2E1', 'IL2RA', 'PTGS2', 'TP53', 'XDH']",...,8,8,0,0,-0.000909,0.000000,-0.000909,1.600000,2.000000,0.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,C0020473,ziprasidone,DB00246,"Hyperlipidemia, combined, 1",Yes,6.202761e-11,33,293,0.000069,"['ADM', 'ADRB1', 'ADRB2', 'ADRB3', 'AGT', 'AGT...",...,29,27,-2,-2,0.017907,0.000916,0.023817,0.878788,1.687500,0.808712
351,C0032285,ziprasidone,DB00246,Pneumonia,Yes,5.466938e-11,44,471,0.000064,"['ADORA2B', 'ADORA3', 'ADRB2', 'C3', 'C3AR1', ...",...,29,27,-2,-2,-0.007354,-0.000115,-0.002993,0.659091,0.870968,0.211877
352,C0686347,ziprasidone,DB00246,Tardive Dyskinesia,Yes,4.269929e-11,18,64,0.000059,"['APOE', 'ARRB2', 'CNR1', 'CYP1A2', 'CYP2D6', ...",...,29,27,-2,-2,0.086875,0.004375,0.113750,1.611111,3.375000,1.763889
353,C0027051,zolmitriptan,DB00315,Myocardial Infarction,Yes,8.673792e-10,29,1019,0.000055,"['ADORA1', 'ADORA3', 'ADRA2B', 'AGT', 'AGTR2',...",...,6,6,0,0,-0.002514,0.000000,-0.002514,0.206897,0.206897,0.000000


In [5]:
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52', '#BCBD22', '#9D755D', '#BAB0AC', '#F7E1A0', '#17BECF', '#1616A7']

adverse_event_lst = ['Sepsis', 'Myocardial Infarction', 'Gastric ulcer', 'Pancreatitis', 'Pneumonia', 'Cerebral Infarction', 'Hemorrhage', 'Deep Vein Thrombosis', 'Edema', 'Tardive Dyskinesia', 'Delirium', 'Hyperlipidemia', 'Thrombocytopenia', 
                     'Peripheral Neuropathy', 'Pulmonary Edema', 'Proteinuria']

color_dict = {adverse_event_lst[i]: colors[i] for i in range(len(colors))}
color_dict

{'Sepsis': '#636EFA',
 'Myocardial Infarction': '#EF553B',
 'Gastric ulcer': '#00CC96',
 'Pancreatitis': '#AB63FA',
 'Pneumonia': '#FFA15A',
 'Cerebral Infarction': '#19D3F3',
 'Hemorrhage': '#FF6692',
 'Deep Vein Thrombosis': '#B6E880',
 'Edema': '#FF97FF',
 'Tardive Dyskinesia': '#FECB52',
 'Delirium': '#BCBD22',
 'Hyperlipidemia': '#9D755D',
 'Thrombocytopenia': '#BAB0AC',
 'Peripheral Neuropathy': '#F7E1A0',
 'Pulmonary Edema': '#17BECF',
 'Proteinuria': '#1616A7'}

In [6]:
benchmark_general_results_folder = "output_benchmark_general_results/"
check_directory_exists(benchmark_general_results_folder)

In [7]:
import plotly.io as pio
pio.renderers.default = "browser"

In [8]:
print("Drug Targets All Same? -", all(test_all_drugs_super_association_df['drug_targets_v1'] == test_all_drugs_super_association_df['drug_targets_v2']))
print("Number of Drug Target lists where contents are all the same:", sum(test_all_drugs_super_association_df['drug_targets_v1'] == test_all_drugs_super_association_df['drug_targets_v2']))

print("Number of Drug Associations in Version 1:", sum(test_all_drugs_super_association_df['association_exists_v1']=="Yes"))
print("Number of Drug Associations in Version 2:", sum(test_all_drugs_super_association_df['association_exists_v2']=="Yes"))
    
temp = test_all_drugs_super_association_df[test_all_drugs_super_association_df['association_exists_v1'] == "Yes"]
temp = temp[temp['association_exists_v2'] == "Yes"]
print("Number of Drug Associations in Version Both Versions:", len(temp))

temp = len(test_all_drugs_super_association_df["drug"].unique())
print("Number of Unique Drugs:", temp)

temp = len(test_all_drugs_super_association_df["cui-drug"].unique())
print("Number of Unique (CUI, Drug) pairs:", temp)

temp = len(test_all_drugs_super_association_df["cui"].unique())
print("Number of Unique CUIs:", temp)

temp = len(test_all_drugs_super_association_df["phenotype_v1"].unique())
print("Number of Unique phenotype_v1:", temp)

temp = len(test_all_drugs_super_association_df["phenotype_v2"].unique())
print("Number of Unique phenotype_v2:", temp)

Drug Targets All Same? - False
Number of Drug Target lists where contents are all the same: 27
Number of Drug Associations in Version 1: 355
Number of Drug Associations in Version 2: 355
Number of Drug Associations in Version Both Versions: 355
Number of Unique Drugs: 195
Number of Unique (CUI, Drug) pairs: 355
Number of Unique CUIs: 16
Number of Unique phenotype_v1: 16
Number of Unique phenotype_v2: 16


In [9]:
import plotly.figure_factory as ff

df = test_all_drugs_super_association_df.copy(deep=True)

v1 = df["-log(benjamini_hochberg_v1)"]
v2 = df["-log(benjamini_hochberg_v2)"]
hist_data = [v1, v2]
group_labels = ['Version 1', 'Verison 2']
fig = ff.create_distplot(hist_data, group_labels, bin_size=0.1, show_rug=False, colors=["#FF5557", "#22D646"])
graph_title = "Distribution of -Log(Benjamini-Hochberg) between PathFX Versions"
fig.update_layout(title_text=graph_title)
fig.update_xaxes(title_text="-log(Benjamini-Hochberg)")
fig.update_yaxes(title_text="Probablity Density")
fig.update_layout(legend_title_text='PathFX Version')
fig.update_layout(barmode='overlay')
fig.update_traces(opacity=0.6)
fig.update_layout(showlegend=True, width=2200, height=1000)
fig.update_layout(xaxis_range=[7,13])
fig.write_image(benchmark_general_results_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "") + ".png")
fig.write_html(benchmark_general_results_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "")  + ".html")
# fig.show()

In [10]:
df = test_all_drugs_super_association_df.copy(deep=True)

phenotype_v2_lst = list(df["phenotype_v2"].unique())

fig = go.Figure()
graph_title = "Distribution of -Log(Benjamini-Hochberg) in PathFX Version 1 for each Adverse Event"
for phenotype in phenotype_v2_lst:
    fig.add_trace(go.Violin(x=df['phenotype_v2'][df['phenotype_v2'] == phenotype], y=df["-log(benjamini_hochberg_v1)"][df['phenotype_v2'] == phenotype], 
                            name=phenotype, box_visible=True, line_color=color_dict[phenotype]))
fig.update_layout(title=graph_title, showlegend=True, autosize=False, width=2200, height=1000)
fig.update_xaxes(title_text="Phenotype")
fig.update_yaxes(title_text="-Log(Benjamini-Hochberg)")
fig.update_layout(legend_title_text='Adverse Event')
fig.write_image(benchmark_general_results_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "") + ".png")
fig.write_html(benchmark_general_results_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "")  + ".html")
# fig.show()

In [11]:
df = test_all_drugs_super_association_df.copy(deep=True)

phenotype_v2_lst = list(df["phenotype_v2"].unique())

fig = go.Figure()
graph_title = "Distribution of -Log(Benjamini-Hochberg) in PathFX Version 2 for each Adverse Event"
for phenotype in phenotype_v2_lst:
    fig.add_trace(go.Violin(x=df['phenotype_v2'][df['phenotype_v2'] == phenotype], y=df["-log(benjamini_hochberg_v2)"][df['phenotype_v2'] == phenotype], 
                            name=phenotype, box_visible=True, line_color=color_dict[phenotype]))
fig.update_layout(title=graph_title, showlegend=True, autosize=False, width=2200, height=1000)
fig.update_xaxes(title_text="Phenotype")
fig.update_yaxes(title_text="-Log(Benjamini-Hochberg)")
fig.update_layout(legend_title_text='Adverse Event')
fig.write_image(benchmark_general_results_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "") + ".png")
fig.write_html(benchmark_general_results_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "")  + ".html")
# fig.show()

## Waterfall plot of version_difference_-log(benjamini_hochberg) 

In [12]:
# all of the cui-drug pairs
df = test_all_drugs_super_association_df.copy(deep=True)

df = df.sort_values(by="version_difference_-log(benjamini_hochberg)", ascending=False)
graph_title = "Difference in -Log(Benjamini-Hochberg) between Version 2 and Version 1 of PathFX for each CUI-Drug Record"
fig = px.bar(df, x="cui-drug", y="version_difference_-log(benjamini_hochberg)", 
             title=graph_title, 
             hover_data=['cui', 'drug', 'drugbank_ID', 'phenotype_v1', 'phenotype_v2', "-log(benjamini_hochberg_v1)",  "-log(benjamini_hochberg_v2)", 
                         "abs_version_difference_-log(benjamini_hochberg)"])
fig.update_xaxes(title_text="CUI-Drug")
fig.update_yaxes(title_text="Difference in -log(Benjamini-Hochberg)")
fig.update_layout(showlegend=False, width=3000, height=2000)
fig.update_traces(
    hovertemplate="<br>".join([
        "CUI = %{customdata[0]}",
        "Phenotype: %{customdata[4]}",
        "Drug: %{customdata[1]}",
        "DrugBank ID: %{customdata[2]}",
        "Version 1 -log(Benjamini-Hochberg): %{customdata[14]}",
        "Version 2 -log(Benjamini-Hochberg): %{customdata[15]}",
        "Difference in -log(Benjamini-Hochberg): %{y}",
        "Abs Difference in -log(Benjamini-Hochberg): %{customdata[16]}"
    ])
)
fig.write_image(benchmark_general_results_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "") + ".png")
fig.write_html(benchmark_general_results_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "")  + ".html")
# fig.show()

# Grouped Drugs
df = test_all_drugs_super_association_df.copy(deep=True)
df = df.groupby("drug").mean().reset_index()
df = df.sort_values(by="version_difference_-log(benjamini_hochberg)", ascending=False)
graph_title = "Mean Difference in -Log(Benjamini-Hochberg) between Version 2 and Version 1 of PathFX for each Drug"
fig = px.bar(df, x="drug", y="version_difference_-log(benjamini_hochberg)", 
             title=graph_title)
fig.update_xaxes(title_text="Drug")
fig.update_yaxes(title_text="Mean Difference in -log(Benjamini-Hochberg)")
fig.update_layout(showlegend=False, width=3000, height=2000)
fig.update_traces(
    hovertemplate="<br>".join([
        "Drug: %{x}",
        "Difference in -log(Benjamini-Hochberg): %{y}"
    ])
)
fig.write_image(benchmark_general_results_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "") + ".png")
fig.write_html(benchmark_general_results_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "")  + ".html")
# fig.show()

# Grouped Phenotypes
df = test_all_drugs_super_association_df.copy(deep=True)
df = df.groupby("phenotype_v2").mean().reset_index()
df = df.sort_values(by="version_difference_-log(benjamini_hochberg)", ascending=False)
graph_title = "Mean Difference in -Log(Benjamini-Hochberg) between Version 2 and Version 1 of PathFX for Phenotype"
fig = px.bar(df, x="phenotype_v2", y="version_difference_-log(benjamini_hochberg)", 
             title=graph_title)
fig.update_xaxes(title_text="Drug")
fig.update_yaxes(title_text="Mean Difference in -log(Benjamini-Hochberg)")
fig.update_layout(showlegend=False, width=3000, height=2000)
fig.update_traces(
    hovertemplate="<br>".join([
        "Phenotype: %{x}",
        "Mean Difference in -log(Benjamini-Hochberg): %{y}"
    ])
)
fig.write_image(benchmark_general_results_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "") + ".png")
fig.write_html(benchmark_general_results_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "")  + ".html")
# fig.show()

## Scatterplots of Benjamini-Hochberg Scores Between PathFX Verison 1 and Version 2

In [13]:
df = test_all_drugs_super_association_df.copy(deep=True)

graph_title = "-log(Benjamini-Hochberg Version 1) vs -log(Benjamini-Hochberg Version 2)"
fig = px.scatter(df, x="-log(benjamini_hochberg_v1)", y="-log(benjamini_hochberg_v2)", 
                 title=graph_title, 
                 color="phenotype_v2", size="abs_version_difference_-log(benjamini_hochberg)",
                 color_discrete_map=color_dict, hover_data=['cui', 'phenotype_v2', 'drug', 'drugbank_ID', 'version_difference_-log(benjamini_hochberg)', 'abs_version_difference_-log(benjamini_hochberg)'])
fig.update_layout(showlegend=True, width=2200, height=1000)
fig.update_xaxes(title_text="-log(Benjamini-Hochberg Version 1)")
fig.update_yaxes(title_text="-log(Benjamini-Hochberg Version 2)")
fig.update_layout(legend_title_text='Phenotype')
fig.update_traces(
    hovertemplate="<br>".join([
        "CUI = %{customdata[0]}",
        "Phenotype: %{customdata[1]}",
        "Drug: %{customdata[2]}",
        "DrugBank ID: %{customdata[3]}",
        "Version 1 -log(Benjamini-Hochberg): %{x}",
        "Version 2 -log(Benjamini-Hochberg): %{y}",
        "Difference in -log(Benjamini-Hochberg): %{customdata[4]}",
        "Abs Difference in -log(Benjamini-Hochberg): %{customdata[5]}"
    ])
)
fig.update_traces(marker=dict(line=dict(width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))
fig.write_image(benchmark_general_results_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "") + ".png")
fig.write_html(benchmark_general_results_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "")  + ".html")
# fig.show()

## Violin Plot for Distribution for Difference in -Log(Benjamini-Hochberg) between PathFX Version 2 and Version 1 

In [14]:
df = test_all_drugs_super_association_df.copy(deep=True)

phenotype_v2_lst = list(df["phenotype_v2"].unique())

fig = go.Figure()
graph_title = "Difference in -Log(Benjamini-Hochberg) between PathFX Version 2 and Version 1 per CUI"
for phenotype in phenotype_v2_lst:
    fig.add_trace(go.Violin(x=df['phenotype_v2'][df['phenotype_v2'] == phenotype], y=df["version_difference_-log(benjamini_hochberg)"][df['phenotype_v2'] == phenotype], 
                            name=phenotype, box_visible=True, line_color=color_dict[phenotype]))
fig.update_layout(title=graph_title, showlegend=True, autosize=False, width=2200, height=1000)
fig.update_xaxes(title_text="Phenotype")
fig.update_yaxes(title_text="Difference in -Log(Benjamini-Hochberg) between PathFX Version 2 and Version 1")
fig.update_layout(legend_title_text='Adverse Event')
fig.write_image(benchmark_general_results_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "") + ".png")
fig.write_html(benchmark_general_results_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "")  + ".html")
# fig.show()

## Table Info on Comparing Difference in -Log(Benjamini-Hochberg) between PathFX Version 2 and Version 1 

In [15]:
df = test_all_drugs_super_association_df.copy(deep=True)

df = df.rename(columns={"phenotype_v2":"Adverse Event", "-log(benjamini_hochberg_v1)":"-Log(Benjamini-Hochberg) PathFXv1", "-log(benjamini_hochberg_v2)":"-Log(Benjamini-Hochberg) PathFXv2"})

bh_all_records_stats_df = df[["-Log(Benjamini-Hochberg) PathFXv1", "-Log(Benjamini-Hochberg) PathFXv2"]].agg(['count', 'mean','std','min','max','sum'])
save_to_csv_file(benchmark_general_results_folder, bh_all_records_stats_df, "bh_all_records_stats_table", input_index=True)
display(bh_all_records_stats_df)

bh_per_adverse_event_stats_df = df[['Adverse Event', "-Log(Benjamini-Hochberg) PathFXv1", "-Log(Benjamini-Hochberg) PathFXv2"]].groupby("Adverse Event").agg(['count','mean','std','min','max','sum'])
save_to_csv_file(benchmark_general_results_folder, bh_per_adverse_event_stats_df, "bh_per_adverse_event_stats_table", input_index=True)
display(bh_per_adverse_event_stats_df)

,-Log(Benjamini-Hochberg) PathFXv1,-Log(Benjamini-Hochberg) PathFXv2
count,355.000000,355.000000
mean,9.456880,9.107798
std,0.589997,0.742159
min,7.244228,7.265256
max,10.378329,12.795525
sum,3357.192415,3233.268339


-Log(Benjamini-Hochberg) PathFXv1                      \
                                                  count      mean       std   
Adverse Event                                                                 
Cerebral Infarction                                   5  9.381406  0.328028   
Deep Vein Thrombosis                                  2  9.211272  0.978045   
Delirium                                             11  8.835049  0.186098   
Edema                                                18  8.882991  0.297118   
Gastric ulcer                                        26  9.499492  0.357292   
Hemorrhage                                           12  9.188641  0.643119   
Hyperlipidemia                                       19  9.541739  0.651264   
Myocardial Infarction                                80  9.687809  0.395735   
Pancreatitis                                         24  8.845175  0.545896   
Peripheral Neuropathy                                 1  8.630963       NaN   
Pneumonia                                            70  9.770840  0.552200   
Proteinuria                                           4  9.208771  0.254004   
Pulmonary Edema                                       3  8.355609  0.246931   
Sepsis                                               33  9.665936  0.370007   
Tardive Dyskinesia                                   24  9.757226  0.260899   
Thrombocytopenia                                     23  8.751742  0.562202   

                                                        \
                            min        max         sum   
Adverse Event                                            
Cerebral Infarction    9.056444   9.843200   46.907030   
Deep Vein Thrombosis   8.519690   9.902854   18.422545   
Delirium               8.617546   9.269498   97.185539   
Edema                  8.360530   9.656958  159.893830   
Gastric ulcer          8.281086  10.009153  246.986801   
Hemorrhage             8.312995   9.922575  110.263690   
Hyperlipidemia         7.656013  10.105494  181.293043   
Myocardial Infarction  7.576678  10.297269  775.024723   
Pancreatitis           7.244228   9.306786  212.284205   
Peripheral Neuropathy  8.630963   8.630963    8.630963   
Pneumonia              7.950614  10.378329  683.958774   
Proteinuria            8.981473   9.471755   36.835084   
Pulmonary Edema        8.080867   8.559030   25.066826   
Sepsis                 8.187692  10.220459  318.975882   
Tardive Dyskinesia     9.061635  10.293519  234.173426   
Thrombocytopenia       7.540734  10.112108  201.290055   

                      -Log(Benjamini-Hochberg) PathFXv2                      \
                                                  count      mean       std   
Adverse Event                                                                 
Cerebral Infarction                                   5  8.750076  0.371986   
Deep Vein Thrombosis                                  2  9.267594  1.342812   
Delirium                                             11  8.790180  0.297606   
Edema                                                18  8.621332  0.266830   
Gastric ulcer                                        26  9.385711  0.236743   
Hemorrhage                                           12  8.813575  0.753574   
Hyperlipidemia                                       19  9.238290  0.640571   
Myocardial Infarction                                80  8.963650  0.351763   
Pancreatitis                                         24  8.491949  0.435240   
Peripheral Neuropathy                                 1  8.030082       NaN   
Pneumonia                                            70  9.746301  0.984377   
Proteinuria                                           4  8.375066  0.166715   
Pulmonary Edema                                       3  8.699391  0.479144   
Sepsis                                               33  9.407381  0.795628   
Tardive Dyskinesia                                   24  9.006102  0.26817

In [16]:
df = test_all_drugs_super_association_df.copy(deep=True)

df = df.rename(columns={"phenotype_v2":"Adverse Event", "version_difference_-log(benjamini_hochberg)":"Difference in -Log(Benjamini-Hochberg) Between PathFX Versions"})

diff_bh_all_records_stats_df = df[["Difference in -Log(Benjamini-Hochberg) Between PathFX Versions"]].agg(['count', 'mean', 'std', 'min', 'max', 'sum'])
save_to_csv_file(benchmark_general_results_folder, diff_bh_all_records_stats_df, "diff_bh_all_records_stats_table", input_index=True)
display(diff_bh_all_records_stats_df)

diff_bh_per_adverse_event_stats_df = df[['Adverse Event', "Difference in -Log(Benjamini-Hochberg) Between PathFX Versions"]].groupby("Adverse Event").agg(['count','mean','std','min','max','sum'])
save_to_csv_file(benchmark_general_results_folder, diff_bh_per_adverse_event_stats_df, "diff_bh_per_adverse_event_stats_table", input_index=True)
display(diff_bh_per_adverse_event_stats_df)

,Difference in -Log(Benjamini-Hochberg) Between PathFX Versions
count,355.000000
mean,-0.349082
std,0.597755
min,-1.993922
max,2.690201
sum,-123.924075


Difference in -Log(Benjamini-Hochberg) Between PathFX Versions  \
                                                                               count   
Adverse Event                                                                          
Cerebral Infarction                                                    5               
Deep Vein Thrombosis                                                   2               
Delirium                                                              11               
Edema                                                                 18               
Gastric ulcer                                                         26               
Hemorrhage                                                            12               
Hyperlipidemia                                                        19               
Myocardial Infarction                                                 80               
Pancreatitis                                                          24               
Peripheral Neuropathy                                                  1               
Pneumonia                                                             70               
Proteinuria                                                            4               
Pulmonary Edema                                                        3               
Sepsis                                                                33               
Tardive Dyskinesia                                                    24               
Thrombocytopenia                                                      23               

                                                                          
                           mean       std       min       max        sum  
Adverse Event                                                             
Cerebral Infarction   -0.631330  0.104205 -0.715941 -0.491164  -3.156649  
Deep Vein Thrombosis   0.056322  0.364767 -0.201607  0.314251   0.112644  
Delirium              -0.044869  0.205156 -0.324484  0.241840  -0.493563  
Edema                 -0.261659  0.366237 -0.999253  0.245765  -4.709855  
Gastric ulcer         -0.113782  0.336602 -0.873747  0.783856  -2.958323  
Hemorrhage            -0.375066  0.433930 -1.140689  0.180825  -4.500789  
Hyperlipidemia        -0.303449  0.258853 -0.723818  0.190676  -5.765537  
Myocardial Infarction -0.724159  0.388938 -1.993922  0.981862 -57.932717  
Pancreatitis          -0.353226  0.444612 -1.129339  0.510826  -8.477427  
Peripheral Neuropathy -0.600881       NaN -0.600881 -0.600881  -0.600881  
Pneumonia             -0.024539  0.796654 -0.961287  2.690201  -1.717720  
Proteinuria           -0.833705  0.393040 -1.340931 -0.494155  -3.334819  
Pulmonary Edema        0.343783  0.607894 -0.356841  0.731233   1.031348  
Sepsis                -0.258555  0.701485 -1.901196  2.580385  -8.532302  
Tardive Dyskinesia    -0.751125  0.239460 -1.176207 -0.208967 -18.026990  
Thrombocytopenia      -0.211326  0.671401 -1.978819  0.794301  -4.860497

In [17]:
df = test_all_drugs_super_association_df.copy(deep=True)

df["Frequency PathFX Version 1 -log(BH) > PathFX Version 2 -log(BH)"] = df["version_difference_-log(benjamini_hochberg)"] < 0
df["Frequency PathFX Version 1 -log(BH) = PathFX Version 2 -log(BH)"] = df["version_difference_-log(benjamini_hochberg)"] == 0
df["Frequency PathFX Version 1 -log(BH) < PathFX Version 2 -log(BH)"] = df["version_difference_-log(benjamini_hochberg)"] > 0

count_bh_comp_df = df[["Frequency PathFX Version 1 -log(BH) > PathFX Version 2 -log(BH)", 
                       "Frequency PathFX Version 1 -log(BH) = PathFX Version 2 -log(BH)", 
                       "Frequency PathFX Version 1 -log(BH) < PathFX Version 2 -log(BH)"]].agg(['count', 'sum'])
bh_inequa_comp_df = count_bh_comp_df.T

save_to_csv_file(benchmark_general_results_folder, bh_inequa_comp_df, "bh_inequa_comp_table", input_index=False)

bh_inequa_comp_df

,count,sum
Frequency PathFX Version 1 -log(BH) > PathFX Version 2 -log(BH),355,287
Frequency PathFX Version 1 -log(BH) = PathFX Version 2 -log(BH),355,0
Frequency PathFX Version 1 -log(BH) < PathFX Version 2 -log(BH),355,68


In [18]:
df = test_all_drugs_super_association_df.copy(deep=True)

df = df.rename(columns={"phenotype_v2":"Adverse Event"})

df["Frequency PathFX Version 1 -log(BH) > PathFX Version 2 -log(BH)"] = df["version_difference_-log(benjamini_hochberg)"] < 0
df["Frequency PathFX Version 1 -log(BH) = PathFX Version 2 -log(BH)"] = df["version_difference_-log(benjamini_hochberg)"] == 0
df["Frequency PathFX Version 1 -log(BH) < PathFX Version 2 -log(BH)"] = df["version_difference_-log(benjamini_hochberg)"] > 0

count_bh_comp_df = df[["Adverse Event",
                       "Frequency PathFX Version 1 -log(BH) > PathFX Version 2 -log(BH)", 
                       "Frequency PathFX Version 1 -log(BH) = PathFX Version 2 -log(BH)", 
                       "Frequency PathFX Version 1 -log(BH) < PathFX Version 2 -log(BH)"]].groupby("Adverse Event").sum()
bh_inequa_adverse_event_comp_df = count_bh_comp_df

save_to_csv_file(benchmark_general_results_folder, bh_inequa_adverse_event_comp_df, "bh_inequa_adverse_event_comp_table", input_index=False)

bh_inequa_adverse_event_comp_df

,Frequency PathFX Version 1 -log(BH) > PathFX Version 2 -log(BH),Frequency PathFX Version 1 -log(BH) = PathFX Version 2 -log(BH),Frequency PathFX Version 1 -log(BH) < PathFX Version 2 -log(BH)
Adverse Event,,,
Cerebral Infarction,5,0,0
Deep Vein Thrombosis,1,0,1
Delirium,6,0,5
Edema,13,0,5
Gastric ulcer,19,0,7
Hemorrhage,9,0,3
Hyperlipidemia,17,0,2
Myocardial Infarction,78,0,2
Pancreatitis,19,0,5


## Drug Content

In [19]:
df = test_all_drugs_super_association_df.copy(deep=True)

drug_frequency_count_df = df.groupby("drug").count()
drug_frequency_count_df = drug_frequency_count_df[["cui"]]
drug_frequency_count_df.columns = ["Count"]
drug_frequency_count_df = drug_frequency_count_df.reset_index()
drug_frequency_count_df = drug_frequency_count_df.sort_values(by=["Count", "drug"], ascending=[False, True])

save_to_csv_file(benchmark_general_results_folder, drug_frequency_count_df, "drug_frequency_count_table", input_index=False)

drug_frequency_count_df

,drug,Count
12,aripiprazole,6
25,caffeine,6
33,clozapine,6
116,olanzapine,6
131,pergolide,6
...,...,...
186,trihexyphenidyl,1
188,triptorelin,1
189,tropicamide,1
191,warfarin,1


In [20]:
drug_frequency_count_df.head(25)

,drug,Count
12,aripiprazole,6
25,caffeine,6
33,clozapine,6
116,olanzapine,6
131,pergolide,6
4,alteplase,5
193,ziprasidone,5
7,amitriptyline,4
11,aprotinin,4
14,atropine,4


In [21]:
df = test_all_drugs_super_association_df.copy(deep=True)
bechmarked_drug_stats_df = df.groupby("drug").mean()

save_to_csv_file(benchmark_general_results_folder, bechmarked_drug_stats_df, "bechmarked_drug_stats_table", input_index=False)

bechmarked_drug_stats_df

,probability_v1,assoc_in_neigh_v1,assoc_in_intom_v1,benjamini_hochberg_v1,probability_v2,assoc_in_neigh_v2,assoc_in_intom_v2,benjamini_hochberg_v2,neigh_genes_v1_and_intom_genes_v1_intersection_size,neigh_genes_v1_and_intom_genes_v1_union_size,...,drug_targets_v1_size,drug_targets_v2_size,version_difference_drug_target_gene_lst,version_difference_unique_drug_target_gene_lst,version_difference_drug_target_gene_lst_percentage,version_difference_unqiue_drug_target_gene_lst_percentage,version_difference_intersecting_drug_target_gene_lst_percentage,ratio_drug_target_gene_over_neigh_gene_lst_v1_size,ratio_drug_target_gene_over_neigh_gene_lst_v2_size,version_difference_ratio_drug_target_gene_over_neigh_gene_lst_size
drug,,,,,,,,,,,,,,,,,,,,,
abatacept,1.079882e-06,5.000000,602.000000,0.000060,3.991105e-05,3.000000,550.000000,0.000317,5.000000,602.000000,...,2.0,2.0,0.0,0.0,0.000314,0.000000,0.000314,0.400000,0.666667,0.266667
acamprosate,3.365007e-09,14.500000,567.500000,0.000061,2.048295e-06,11.500000,936.000000,0.000096,14.500000,567.500000,...,1.0,2.0,1.0,1.0,0.006400,0.000799,0.000799,0.086310,0.225490,0.139181
adenosine,1.735121e-10,36.000000,1019.000000,0.000064,9.629507e-07,35.000000,1778.000000,0.000117,36.000000,1019.000000,...,6.0,6.0,0.0,0.0,-0.002514,0.000000,-0.002514,0.166667,0.171429,0.004762
aldesleukin,7.598317e-05,9.000000,374.000000,0.000176,1.181727e-04,6.000000,392.000000,0.000162,9.000000,374.000000,...,8.0,8.0,0.0,0.0,-0.000915,0.000000,-0.000915,1.107692,1.500000,0.392308
alteplase,7.368401e-10,8.400000,420.800000,0.000062,1.835830e-06,5.000000,551.000000,0.000138,8.400000,420.800000,...,4.0,4.0,0.0,0.0,0.011542,0.000000,0.011542,0.509158,0.874286,0.365128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
vandetanib,5.651149e-07,17.666667,246.333333,0.000139,4.655093e-05,12.333333,275.666667,0.000230,17.666667,246.333333,...,11.0,13.0,2.0,2.0,0.004182,-0.000623,-0.003427,0.623094,1.064574,0.441481
warfarin,1.014538e-09,17.000000,293.000000,0.000043,3.425659e-08,11.000000,231.000000,0.000053,17.000000,293.000000,...,10.0,10.0,0.0,0.0,0.009160,0.000916,0.008244,0.588235,0.909091,0.320856
zidovudine,7.657270e-09,18.000000,293.000000,0.000043,1.487053e-06,10.000000,231.000000,0.000055,18.000000,293.000000,...,19.0,19.0,0.0,0.0,0.017405,0.001832,0.015573,1.055556,1.900000,0.844444


In [22]:
df = test_all_drugs_super_association_df.copy(deep=True)

df["Frequency PathFX Version 1 -log(BH) > PathFX Version 2 -log(BH)"] = df["version_difference_-log(benjamini_hochberg)"] < 0
df["Frequency PathFX Version 1 -log(BH) = PathFX Version 2 -log(BH)"] = df["version_difference_-log(benjamini_hochberg)"] == 0
df["Frequency PathFX Version 1 -log(BH) < PathFX Version 2 -log(BH)"] = df["version_difference_-log(benjamini_hochberg)"] > 0

count_drug_bh_df = df.groupby("drug").sum()

count_drug_bh_df = count_drug_bh_df[["Frequency PathFX Version 1 -log(BH) > PathFX Version 2 -log(BH)", 
                  "Frequency PathFX Version 1 -log(BH) = PathFX Version 2 -log(BH)", 
                  "Frequency PathFX Version 1 -log(BH) < PathFX Version 2 -log(BH)"]]
count_drug_bh_df = count_drug_bh_df.reset_index()

drug_bh_assoc_version_df = drug_frequency_count_df.merge(count_drug_bh_df, left_on="drug", right_on="drug")

save_to_csv_file(benchmark_general_results_folder, drug_bh_assoc_version_df, "drug_bh_assoc_version_table", input_index=False)

drug_bh_assoc_version_df

,drug,Count,Frequency PathFX Version 1 -log(BH) > PathFX Version 2 -log(BH),Frequency PathFX Version 1 -log(BH) = PathFX Version 2 -log(BH),Frequency PathFX Version 1 -log(BH) < PathFX Version 2 -log(BH)
0,aripiprazole,6,5,0,1
1,caffeine,6,6,0,0
2,clozapine,6,6,0,0
3,olanzapine,6,6,0,0
4,pergolide,6,4,0,2
...,...,...,...,...,...
190,trihexyphenidyl,1,1,0,0
191,triptorelin,1,1,0,0
192,tropicamide,1,1,0,0
193,warfarin,1,1,0,0


In [23]:
drug_bh_assoc_version_df.head(20)

,drug,Count,Frequency PathFX Version 1 -log(BH) > PathFX Version 2 -log(BH),Frequency PathFX Version 1 -log(BH) = PathFX Version 2 -log(BH),Frequency PathFX Version 1 -log(BH) < PathFX Version 2 -log(BH)
0,aripiprazole,6,5,0,1
1,caffeine,6,6,0,0
2,clozapine,6,6,0,0
3,olanzapine,6,6,0,0
4,pergolide,6,4,0,2
5,alteplase,5,4,0,1
6,ziprasidone,5,5,0,0
7,amitriptyline,4,4,0,0
8,aprotinin,4,4,0,0
9,atropine,4,4,0,0


In [24]:
temp_df = drug_bh_assoc_version_df.melt(["drug", "Count"])
temp_df

,drug,Count,variable,value
0,aripiprazole,6,Frequency PathFX Version 1 -log(BH) > PathFX V...,5
1,caffeine,6,Frequency PathFX Version 1 -log(BH) > PathFX V...,6
2,clozapine,6,Frequency PathFX Version 1 -log(BH) > PathFX V...,6
3,olanzapine,6,Frequency PathFX Version 1 -log(BH) > PathFX V...,6
4,pergolide,6,Frequency PathFX Version 1 -log(BH) > PathFX V...,4
...,...,...,...,...
580,trihexyphenidyl,1,Frequency PathFX Version 1 -log(BH) < PathFX V...,0
581,triptorelin,1,Frequency PathFX Version 1 -log(BH) < PathFX V...,0
582,tropicamide,1,Frequency PathFX Version 1 -log(BH) < PathFX V...,0
583,warfarin,1,Frequency PathFX Version 1 -log(BH) < PathFX V...,0


In [25]:
temp_df = temp_df.sort_values(by="Count", ascending=False)

graph_title = "-log(BH) value comparison for each drug in PathFX Version 1 vs PathFX Version 2"
fig = px.bar(data_frame=temp_df, x="value", y="drug", color="variable",
             title=graph_title).update_yaxes(categoryorder="total ascending")
fig.update_layout(showlegend=True, width=2800, height=3000)
fig.update_xaxes(title_text="Count")
fig.update_yaxes(title_text="Drug")
fig.update_traces(
    hovertemplate="<br>".join([
        "Drug: %{y}",
        "Count: %{x}"
    ])
)
fig.write_image(benchmark_general_results_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "") + ".png")
fig.write_html(benchmark_general_results_folder + graph_title.replace(" ", "_").replace("(", "_").replace(")", "")  + ".html")
# fig.show()